# Imports

In [1]:
from collections import Counter

from tqdm import tqdm

from pfc_packages.gpf_slr import refine_ground_plane, scan_line_run_clustering
from pfc_packages.utils import (
    Dataset,
    PointCloudVisualizer,
    get_statistics,
    run_viz,
    save_clusters,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Hyperparameters

In [2]:
# --- Hyperparameters for Ground Plane Fitting (GPF) ---
NUM_LOWEST_POINTS = 10000  # Number of lowest elevation points used to estimate initial ground seed (LPR)
NUM_ITERATIONS = 5  # Number of iterations for plane refinement in GPF
SEED_HEIGHT_THRESHOLD = 0.4  # Max height above LPR to consider a point as a ground seed
PLANE_DISTANCE_THRESHOLD = 0.2  # Max distance from plane to classify a point as ground

# --- Parameters for Scan Line Run (SLR) clustering ---
SLR_RUN_DISTANCE_THRESHOLD = 0.2  # Max distance between consecutive points in a scanline to form a run
SLR_MERGE_THRESHOLD = 1.0  # Max distance between runs in adjacent scanlines to be considered the same cluster

# Run Example

In [3]:
dataset = Dataset("../../datasets/semantic-kitti", load_cluster=True)
point_cloud = dataset[0]

# point_cloud, normal_d_tuple = refine_ground_plane(point_cloud,  # type: ignore
#                                              num_points=NUM_LOWEST_POINTS,
#                                              height_threshold=SEED_HEIGHT_THRESHOLD,
#                                              distance_threshold=PLANE_DISTANCE_THRESHOLD,
#                                              num_iterations=NUM_ITERATIONS)

# point_cloud = scan_line_run_clustering(point_cloud, SLR_RUN_DISTANCE_THRESHOLD, SLR_MERGE_THRESHOLD)

# PointCloudVisualizer().show(
#     point_cloud=point_cloud, show_true_label=False, show_plane=False, show_grid=False
# )

In [ ]:
dataset = Dataset("../../datasets/semantic-kitti")
OUTPUT_DIR = "../../datasets/semantic-kitti/data_odometry_clusters/dataset/sequences"

resumo_por_frame = []
combinacoes_geral = Counter()

for idx in tqdm(range(len(dataset)), desc="Processando frames"):
    point_cloud = dataset[idx]

    point_cloud, normal_d_tuple = refine_ground_plane(
        point_cloud,  # type: ignore
        num_points=NUM_LOWEST_POINTS,
        height_threshold=SEED_HEIGHT_THRESHOLD,
        distance_threshold=PLANE_DISTANCE_THRESHOLD,
        num_iterations=NUM_ITERATIONS,
    )
    point_cloud = scan_line_run_clustering(
        point_cloud, SLR_RUN_DISTANCE_THRESHOLD, SLR_MERGE_THRESHOLD
    )

    estatisticas_do_frame = get_statistics(idx, point_cloud)
    resumo_por_frame.append(estatisticas_do_frame)
    combinacoes_geral += estatisticas_do_frame["combinacoes"]

    # result = dataset.get_last_seq_frame()
    # if result is not None:
    #     seq, frame_idx = result
    #     save_clusters(point_cloud, seq, frame_idx, OUTPUT_DIR)

Processando frames: 100%|██████████| 12097/12097 [2:21:43<00:00,  1.42it/s] 


In [5]:
# from pfc_packages.utils import plot_1
# from pfc_packages.utils import plot_2
# from pfc_packages.utils import plot_3
# from pfc_packages.utils import plot_4
# from pfc_packages.utils import plot_5

# seq = "100"

# plot_1(resumo_por_frame, seq)
# plot_2(resumo_por_frame, seq)
# plot_3(resumo_por_frame, seq)
# plot_4(resumo_por_frame, seq)
# plot_5(combinacoes_geral, seq)